# How to ingest files from Snowflake Stages, infer the schemas, use Cortex LLM functions, and leverage Iceberg tables in Coalesce

In this notebook, you can view the SQL generated to perform all of the actions to build an LLM pipeline in Coalesce. This SQL is automatically generated and managed by Coalesce. Variations of this code are possible depending on the version of Coalesce packages are you using. 

The dataset is a call transcripts dataset that contains calls in multiple different languages. This walkthrough will teach you to ingest the data, using Coalesce Cortex Functions to translate the data, and expose the data to your organization using Iceberg nodes. 

First, we will use an Inferschema node to create a blank table in Snowflake which contains the schema of the file(s) in a stage. 

In [ ]:
-- Anonymous stored procedure block
DECLARE
-- Variables
col_name VARCHAR;

col_type VARCHAR;

create_sql VARCHAR;

final_sql VARCHAR;

-- Cursor to loop through each row in GROUP_LOAD table
res RESULTSET;

select_statement:= 'CREATE OR REPLACE TABLE "DATABASE_NAME"."SCHEMA_NAME".CALL_TRANSCRIPTS
                       USING TEMPLATE (SELECT ARRAY_AGG(OBJECT_CONSTRUCT(*))
                      FROM TABLE(
                        INFER_SCHEMA(
                          LOCATION=>''@"PC_COALESCE_DB"."CALLS".call_transcripts_data_stage'',
                           FILES =>''call_transcripts.csv'',
                          FILE_FORMAT=>''"PC_COALESCE_DB"."CALLS".csvformat''
                        )
                      ))';

BEGIN
EXECUTE IMMEDIATE:select_statement;

-- Return a success message
RETURN 'CALL_TRANSCRIPTS table created successfully.';

END;

Next, we need to load the data from the file(s) in the stage, into the table containing the schema we just created. We can use a Copy Into node for this, which automatically generates the SQL below

In [ ]:
CREATE OR REPLACE table
  "DATABASE_NAME"."SCHEMA_NAME"."CALL_TRANSCRIPTS_LOAD" (
    "DATE_CREATED" DATE,
    "LANGUAGE" VARCHAR (16777216),
    "COUNTRY" VARCHAR (16777216),
    "PRODUCT" VARCHAR (16777216),
    "CATEGORY" VARCHAR (16777216),
    "DAMAGE_TYPE" VARCHAR (16777216),
    "TRANSCRIPT" VARCHAR (16777216),
    "SRC" VARCHAR,
    "LOAD_TIMESTAMP" TIMESTAMP_NTZ,
    "FILENAME" STRING,
    "FILE_ROW_NUMBER" NUMBER,
    "FILE_LAST_MODIFIED" TIMESTAMP_NTZ,
    "SCAN_TIME" TIMESTAMP_NTZ
  );

COPY INTO "DATABASE_NAME"."SCHEMA_NAME"."CALL_TRANSCRIPTS_LOAD" (
  "DATE_CREATED",
  "LANGUAGE",
  "COUNTRY",
  "PRODUCT",
  "CATEGORY",
  "DAMAGE_TYPE",
  "TRANSCRIPT",
  "SRC",
  "LOAD_TIMESTAMP",
  "FILENAME",
  "FILE_ROW_NUMBER",
  "FILE_LAST_MODIFIED",
  "SCAN_TIME"
)
FROM
  (
    SELECT
      $1::DATE AS "DATE_CREATED",
      $2::VARCHAR (16777216) AS "LANGUAGE",
      $3::VARCHAR (16777216) AS "COUNTRY",
      $4::VARCHAR (16777216) AS "PRODUCT",
      $5::VARCHAR (16777216) AS "CATEGORY",
      $6::VARCHAR (16777216) AS "DAMAGE_TYPE",
      $7::VARCHAR (16777216) AS "TRANSCRIPT",
      $1 AS "SRC",
      current_timestamp()::timestamp_ntz AS "LOAD_TIMESTAMP",
      METADATA$FILENAME AS "FILENAME",
      METADATA$FILE_ROW_NUMBER AS "FILE_ROW_NUMBER",
      METADATA$FILE_LAST_MODIFIED AS "FILE_LAST_MODIFIED",
      METADATA$START_SCAN_TIME AS "SCAN_TIME"
    FROM
      '@PC_COALESCE_DB.CALLS.call_transcripts_data_stage'
  ) FILES = ('call_transcripts.csv') FILE_FORMAT = (
    TYPE = CSV RECORD_DELIMITER = '
' FIELD_DELIMITER = ',' FIELD_OPTIONALLY_ENCLOSED_BY = '\042' SKIP_HEADER = 1
  );

Now that we have a table that is populated in Snowflake, we can start building out a data pipeline and transforming this data. The first thing we'll do is split the German and French langauges from the dataset into their own tables. The following SQL is generated by stage nodes in Coalesce, and represents both stages used to split the French and German languages into their own tables. 

In [ ]:
CREATE OR REPLACE TABLE
  "DATABASE_NAME"."SCHEMA_NAME"."STG_CALL_TRANSCRIPTS_GERMAN" (
    "DATE_CREATED" DATE,
    "LANGUAGE" VARCHAR (16777216),
    "COUNTRY" VARCHAR (16777216),
    "PRODUCT" VARCHAR (16777216),
    "CATEGORY" VARCHAR (16777216),
    "DAMAGE_TYPE" VARCHAR (16777216),
    "TRANSCRIPT" VARCHAR (16777216),
    "SRC" VARCHAR,
    "LOAD_TIMESTAMP" TIMESTAMP_NTZ,
    "FILENAME" STRING,
    "FILE_ROW_NUMBER" NUMBER,
    "FILE_LAST_MODIFIED" TIMESTAMP_NTZ,
    "SCAN_TIME" TIMESTAMP_NTZ
  );

INSERT INTO
  "DATABASE_NAME"."SCHEMA_NAME"."STG_CALL_TRANSCRIPTS_GERMAN" (
    "DATE_CREATED",
    "LANGUAGE",
    "COUNTRY",
    "PRODUCT",
    "CATEGORY",
    "DAMAGE_TYPE",
    "TRANSCRIPT",
    "SRC",
    "LOAD_TIMESTAMP",
    "FILENAME",
    "FILE_ROW_NUMBER",
    "FILE_LAST_MODIFIED",
    "SCAN_TIME"
  )
SELECT
  "CALL_TRANSCRIPTS_LOAD"."DATE_CREATED" AS "DATE_CREATED",
  "CALL_TRANSCRIPTS_LOAD"."LANGUAGE" AS "LANGUAGE",
  "CALL_TRANSCRIPTS_LOAD"."COUNTRY" AS "COUNTRY",
  "CALL_TRANSCRIPTS_LOAD"."PRODUCT" AS "PRODUCT",
  "CALL_TRANSCRIPTS_LOAD"."CATEGORY" AS "CATEGORY",
  "CALL_TRANSCRIPTS_LOAD"."DAMAGE_TYPE" AS "DAMAGE_TYPE",
  "CALL_TRANSCRIPTS_LOAD"."TRANSCRIPT" AS "TRANSCRIPT",
  "CALL_TRANSCRIPTS_LOAD"."SRC" AS "SRC",
  "CALL_TRANSCRIPTS_LOAD"."LOAD_TIMESTAMP" AS "LOAD_TIMESTAMP",
  "CALL_TRANSCRIPTS_LOAD"."FILENAME" AS "FILENAME",
  "CALL_TRANSCRIPTS_LOAD"."FILE_ROW_NUMBER" AS "FILE_ROW_NUMBER",
  "CALL_TRANSCRIPTS_LOAD"."FILE_LAST_MODIFIED" AS "FILE_LAST_MODIFIED",
  "CALL_TRANSCRIPTS_LOAD"."SCAN_TIME" AS "SCAN_TIME"
FROM
  "DATABASE_NAME"."SCHEMA_NAME"."CALL_TRANSCRIPTS_LOAD" "CALL_TRANSCRIPTS_LOAD"
WHERE
  "LANGUAGE" = 'German';

CREATE OR REPLACE TABLE
 "DATABASE_NAME"."SCHEMA_NAME"."STG_CALL_TRANSCRIPTS_FRENCH" (
    "DATE_CREATED" DATE,
    "LANGUAGE" VARCHAR (16777216),
    "COUNTRY" VARCHAR (16777216),
    "PRODUCT" VARCHAR (16777216),
    "CATEGORY" VARCHAR (16777216),
    "DAMAGE_TYPE" VARCHAR (16777216),
    "TRANSCRIPT" VARCHAR (16777216),
    "SRC" VARCHAR,
    "LOAD_TIMESTAMP" TIMESTAMP_NTZ,
    "FILENAME" STRING,
    "FILE_ROW_NUMBER" NUMBER,
    "FILE_LAST_MODIFIED" TIMESTAMP_NTZ,
    "SCAN_TIME" TIMESTAMP_NTZ
  );


INSERT INTO
  "DATABASE_NAME"."SCHEMA_NAME"."STG_CALL_TRANSCRIPTS_FRENCH" (
    "DATE_CREATED",
    "LANGUAGE",
    "COUNTRY",
    "PRODUCT",
    "CATEGORY",
    "DAMAGE_TYPE",
    "TRANSCRIPT",
    "SRC",
    "LOAD_TIMESTAMP",
    "FILENAME",
    "FILE_ROW_NUMBER",
    "FILE_LAST_MODIFIED",
    "SCAN_TIME"
  )
SELECT
  "CALL_TRANSCRIPTS_LOAD"."DATE_CREATED" AS "DATE_CREATED",
  "CALL_TRANSCRIPTS_LOAD"."LANGUAGE" AS "LANGUAGE",
  "CALL_TRANSCRIPTS_LOAD"."COUNTRY" AS "COUNTRY",
  "CALL_TRANSCRIPTS_LOAD"."PRODUCT" AS "PRODUCT",
  "CALL_TRANSCRIPTS_LOAD"."CATEGORY" AS "CATEGORY",
  "CALL_TRANSCRIPTS_LOAD"."DAMAGE_TYPE" AS "DAMAGE_TYPE",
  "CALL_TRANSCRIPTS_LOAD"."TRANSCRIPT" AS "TRANSCRIPT",
  "CALL_TRANSCRIPTS_LOAD"."SRC" AS "SRC",
  "CALL_TRANSCRIPTS_LOAD"."LOAD_TIMESTAMP" AS "LOAD_TIMESTAMP",
  "CALL_TRANSCRIPTS_LOAD"."FILENAME" AS "FILENAME",
  "CALL_TRANSCRIPTS_LOAD"."FILE_ROW_NUMBER" AS "FILE_ROW_NUMBER",
  "CALL_TRANSCRIPTS_LOAD"."FILE_LAST_MODIFIED" AS "FILE_LAST_MODIFIED",
  "CALL_TRANSCRIPTS_LOAD"."SCAN_TIME" AS "SCAN_TIME"
FROM
  "DATABASE_NAME"."SCHEMA_NAME"."CALL_TRANSCRIPTS_LOAD" "CALL_TRANSCRIPTS_LOAD"
WHERE
  "LANGUAGE" IN ('English', 'French');

Now that we have the call transcripts of each language in their own tables, we can use Coalesce Cortex Function nodes, which leverage Snowflake Cortex functions, to translate the data into English. Coalesce automatically generates the code below to perform this action.

In [ ]:
CREATE OR REPLACE TABLE
   "DATABASE_NAME"."SCHEMA_NAME"."LLM__CALL_TRANSCRIPTS_GERMAN" (
    "DATE_CREATED" DATE,
    "LANGUAGE" VARCHAR (16777216),
    "COUNTRY" VARCHAR (16777216),
    "PRODUCT" VARCHAR (16777216),
    "CATEGORY" VARCHAR (16777216),
    "DAMAGE_TYPE" VARCHAR (16777216),
    "TRANSCRIPT" VARCHAR (16777216),
    "SRC" VARCHAR,
    "LOAD_TIMESTAMP" TIMESTAMP_NTZ,
    "FILENAME" STRING,
    "FILE_ROW_NUMBER" NUMBER,
    "FILE_LAST_MODIFIED" TIMESTAMP_NTZ,
    "SCAN_TIME" TIMESTAMP_NTZ
  );

INSERT INTO
   "DATABASE_NAME"."SCHEMA_NAME"."LLM__CALL_TRANSCRIPTS_GERMAN" (
    "DATE_CREATED",
    "LANGUAGE",
    "COUNTRY",
    "PRODUCT",
    "CATEGORY",
    "DAMAGE_TYPE",
    "TRANSCRIPT",
    "SRC",
    "LOAD_TIMESTAMP",
    "FILENAME",
    "FILE_ROW_NUMBER",
    "FILE_LAST_MODIFIED",
    "SCAN_TIME"
  )
SELECT
  "STG_CALL_TRANSCRIPTS_GERMAN"."DATE_CREATED" AS "DATE_CREATED",
  "STG_CALL_TRANSCRIPTS_GERMAN"."LANGUAGE" AS "LANGUAGE",
  "STG_CALL_TRANSCRIPTS_GERMAN"."COUNTRY" AS "COUNTRY",
  "STG_CALL_TRANSCRIPTS_GERMAN"."PRODUCT" AS "PRODUCT",
  "STG_CALL_TRANSCRIPTS_GERMAN"."CATEGORY" AS "CATEGORY",
  "STG_CALL_TRANSCRIPTS_GERMAN"."DAMAGE_TYPE" AS "DAMAGE_TYPE",
  SNOWFLAKE.CORTEX.TRANSLATE (
    "STG_CALL_TRANSCRIPTS_GERMAN"."TRANSCRIPT",
    'de',
    'en'
  ) AS "TRANSCRIPT",
  "STG_CALL_TRANSCRIPTS_GERMAN"."SRC" AS "SRC",
  "STG_CALL_TRANSCRIPTS_GERMAN"."LOAD_TIMESTAMP" AS "LOAD_TIMESTAMP",
  "STG_CALL_TRANSCRIPTS_GERMAN"."FILENAME" AS "FILENAME",
  "STG_CALL_TRANSCRIPTS_GERMAN"."FILE_ROW_NUMBER" AS "FILE_ROW_NUMBER",
  "STG_CALL_TRANSCRIPTS_GERMAN"."FILE_LAST_MODIFIED" AS "FILE_LAST_MODIFIED",
  "STG_CALL_TRANSCRIPTS_GERMAN"."SCAN_TIME" AS "SCAN_TIME"
FROM
   "DATABASE_NAME"."SCHEMA_NAME"."STG_CALL_TRANSCRIPTS_GERMAN" "STG_CALL_TRANSCRIPTS_GERMAN";


CREATE OR REPLACE TABLE
   "DATABASE_NAME"."SCHEMA_NAME"."LLM__CALL_TRANSCRIPTS_FRENCH" (
    "DATE_CREATED" DATE,
    "LANGUAGE" VARCHAR (16777216),
    "COUNTRY" VARCHAR (16777216),
    "PRODUCT" VARCHAR (16777216),
    "CATEGORY" VARCHAR (16777216),
    "DAMAGE_TYPE" VARCHAR (16777216),
    "TRANSCRIPT" VARCHAR (16777216),
    "SRC" VARCHAR,
    "LOAD_TIMESTAMP" TIMESTAMP_NTZ,
    "FILENAME" STRING,
    "FILE_ROW_NUMBER" NUMBER,
    "FILE_LAST_MODIFIED" TIMESTAMP_NTZ,
    "SCAN_TIME" TIMESTAMP_NTZ
  );


INSERT INTO
   "DATABASE_NAME"."SCHEMA_NAME"."LLM__CALL_TRANSCRIPTS_FRENCH" (
    "DATE_CREATED",
    "LANGUAGE",
    "COUNTRY",
    "PRODUCT",
    "CATEGORY",
    "DAMAGE_TYPE",
    "TRANSCRIPT",
    "SRC",
    "LOAD_TIMESTAMP",
    "FILENAME",
    "FILE_ROW_NUMBER",
    "FILE_LAST_MODIFIED",
    "SCAN_TIME"
  )
SELECT
  "STG_CALL_TRANSCRIPTS_FRENCH"."DATE_CREATED" AS "DATE_CREATED",
  "STG_CALL_TRANSCRIPTS_FRENCH"."LANGUAGE" AS "LANGUAGE",
  "STG_CALL_TRANSCRIPTS_FRENCH"."COUNTRY" AS "COUNTRY",
  "STG_CALL_TRANSCRIPTS_FRENCH"."PRODUCT" AS "PRODUCT",
  "STG_CALL_TRANSCRIPTS_FRENCH"."CATEGORY" AS "CATEGORY",
  "STG_CALL_TRANSCRIPTS_FRENCH"."DAMAGE_TYPE" AS "DAMAGE_TYPE",
  SNOWFLAKE.CORTEX.TRANSLATE (
    "STG_CALL_TRANSCRIPTS_FRENCH"."TRANSCRIPT",
    'fr',
    'en'
  ) AS "TRANSCRIPT",
  "STG_CALL_TRANSCRIPTS_FRENCH"."SRC" AS "SRC",
  "STG_CALL_TRANSCRIPTS_FRENCH"."LOAD_TIMESTAMP" AS "LOAD_TIMESTAMP",
  "STG_CALL_TRANSCRIPTS_FRENCH"."FILENAME" AS "FILENAME",
  "STG_CALL_TRANSCRIPTS_FRENCH"."FILE_ROW_NUMBER" AS "FILE_ROW_NUMBER",
  "STG_CALL_TRANSCRIPTS_FRENCH"."FILE_LAST_MODIFIED" AS "FILE_LAST_MODIFIED",
  "STG_CALL_TRANSCRIPTS_FRENCH"."SCAN_TIME" AS "SCAN_TIME"
FROM
   "DATABASE_NAME"."SCHEMA_NAME"."STG_CALL_TRANSCRIPTS_FRENCH" "STG_CALL_TRANSCRIPTS_FRENCH";

With the call transcripts now translated, we can unify the two datasets to bring together all of the translated calls into a singular table. We can use a `UNION ALL` for this within a Stage node in Coalesce. The following is the code that is generated by Coalesce to perform this action. 

In [ ]:
CREATE OR REPLACE TABLE
 "DATABASE_NAME"."SCHEMA_NAME"."STG_CALL_TRANSCRIPTS_ALL" (
    "DATE_CREATED" DATE,
    "LANGUAGE" VARCHAR (16777216),
    "COUNTRY" VARCHAR (16777216),
    "PRODUCT" VARCHAR (16777216),
    "CATEGORY" VARCHAR (16777216),
    "DAMAGE_TYPE" VARCHAR (16777216),
    "TRANSCRIPT" VARCHAR (16777216)
  );

INSERT INTO
  "DATABASE_NAME"."SCHEMA_NAME"."STG_CALL_TRANSCRIPTS_ALL" (
    "DATE_CREATED",
    "LANGUAGE",
    "COUNTRY",
    "PRODUCT",
    "CATEGORY",
    "DAMAGE_TYPE",
    "TRANSCRIPT"
  )
SELECT
  "LLM__CALL_TRANSCRIPTS_GERMAN"."DATE_CREATED" AS "DATE_CREATED",
  "LLM__CALL_TRANSCRIPTS_GERMAN"."LANGUAGE" AS "LANGUAGE",
  "LLM__CALL_TRANSCRIPTS_GERMAN"."COUNTRY" AS "COUNTRY",
  "LLM__CALL_TRANSCRIPTS_GERMAN"."PRODUCT" AS "PRODUCT",
  "LLM__CALL_TRANSCRIPTS_GERMAN"."CATEGORY" AS "CATEGORY",
  "LLM__CALL_TRANSCRIPTS_GERMAN"."DAMAGE_TYPE" AS "DAMAGE_TYPE",
  "LLM__CALL_TRANSCRIPTS_GERMAN"."TRANSCRIPT" AS "TRANSCRIPT"
FROM
  "DATABASE_NAME"."SCHEMA_NAME"."LLM__CALL_TRANSCRIPTS_GERMAN" "LLM__CALL_TRANSCRIPTS_GERMAN"
UNION ALL
SELECT
  "LLM__CALL_TRANSCRIPTS_FRENCH"."DATE_CREATED" AS "DATE_CREATED",
  "LLM__CALL_TRANSCRIPTS_FRENCH"."LANGUAGE" AS "LANGUAGE",
  "LLM__CALL_TRANSCRIPTS_FRENCH"."COUNTRY" AS "COUNTRY",
  "LLM__CALL_TRANSCRIPTS_FRENCH"."PRODUCT" AS "PRODUCT",
  "LLM__CALL_TRANSCRIPTS_FRENCH"."CATEGORY" AS "CATEGORY",
  "LLM__CALL_TRANSCRIPTS_FRENCH"."DAMAGE_TYPE" AS "DAMAGE_TYPE",
  "LLM__CALL_TRANSCRIPTS_FRENCH"."TRANSCRIPT" AS "TRANSCRIPT"
FROM
 "DATABASE_NAME"."SCHEMA_NAME"."LLM__CALL_TRANSCRIPTS_FRENCH" "LLM__CALL_TRANSCRIPTS_FRENCH"

With the dataset containing the translated call transcripts unified, we can now use another Cortex Function node to create a sentiment score and extract the customer name from the transcript column. Coalesce automatically generates the following code. 

In [ ]:
CREATE OR REPLACE TABLE
  "DATABASE_NAME"."SCHEMA_NAME"."LLM__CALL_TRANSCRIPTS_ALL" (
    "DATE_CREATED" DATE,
    "LANGUAGE" VARCHAR (16777216),
    "COUNTRY" VARCHAR (16777216),
    "PRODUCT" VARCHAR (16777216),
    "CATEGORY" VARCHAR (16777216),
    "DAMAGE_TYPE" VARCHAR (16777216),
    "TRANSCRIPT_SENTIMENT" VARCHAR (16777216),
    "TRANSCRIPT_CUSTOMER" ARRAY
  );

INSERT INTO
  "DATABASE_NAME"."SCHEMA_NAME"."LLM__CALL_TRANSCRIPTS_ALL" (
    "DATE_CREATED",
    "LANGUAGE",
    "COUNTRY",
    "PRODUCT",
    "CATEGORY",
    "DAMAGE_TYPE",
    "TRANSCRIPT_SENTIMENT",
    "TRANSCRIPT_CUSTOMER"
  )
SELECT
  "STG_CALL_TRANSCRIPTS_ALL"."DATE_CREATED" AS "DATE_CREATED",
  "STG_CALL_TRANSCRIPTS_ALL"."LANGUAGE" AS "LANGUAGE",
  "STG_CALL_TRANSCRIPTS_ALL"."COUNTRY" AS "COUNTRY",
  "STG_CALL_TRANSCRIPTS_ALL"."PRODUCT" AS "PRODUCT",
  "STG_CALL_TRANSCRIPTS_ALL"."CATEGORY" AS "CATEGORY",
  "STG_CALL_TRANSCRIPTS_ALL"."DAMAGE_TYPE" AS "DAMAGE_TYPE",
  SNOWFLAKE.CORTEX.SENTIMENT ("STG_CALL_TRANSCRIPTS_ALL"."TRANSCRIPT") AS "TRANSCRIPT_SENTIMENT",
  SNOWFLAKE.CORTEX.EXTRACT_ANSWER (
    "STG_CALL_TRANSCRIPTS_ALL"."TRANSCRIPT",
    '''who is the customer?'''
  ) AS "TRANSCRIPT_CUSTOMER"
FROM
  "DATABASE_NAME"."SCHEMA_NAME"."STG_CALL_TRANSCRIPTS_ALL" "STG_CALL_TRANSCRIPTS_ALL";

The last step is to expose the new analytical dataset we have created to our object storage so that other systems can work with a single copy of the data we have produced in this pipeline. We can do that with a Snowflake Iceberg table node, which will seamlessly create an Iceberg format table in an S3 bucket (assuming you have already configured an external volume). Coalesce automatically generates the code below. 

In [ ]:
CREATE
OR REPLACE ICEBERG TABLE  "DATABASE_NAME"."SCHEMA_NAME"."ICT_CALL_TRANSCRIPTS_ALL" (
  "DATE_CREATED" DATE,
  "LANGUAGE" STRING,
  "COUNTRY" STRING,
  "PRODUCT" STRING,
  "CATEGORY" STRING,
  "DAMAGE_TYPE" STRING,
  "TRANSCRIPT_SENTIMENT" STRING,
  "CUSTOMER_NAME" STRING
) EXTERNAL_VOLUME = 'iceberg_external_volume' BASE_LOCATION = 'transcriptions_customer' CATALOG = 'SNOWFLAKE';


INSERT INTO
  "DATABASE_NAME"."SCHEMA_NAME"."ICT_CALL_TRANSCRIPTS_ALL" (
    "DATE_CREATED",
    "LANGUAGE",
    "COUNTRY",
    "PRODUCT",
    "CATEGORY",
    "DAMAGE_TYPE",
    "TRANSCRIPT_SENTIMENT",
    "CUSTOMER_NAME"
  ) (
    SELECT
      "LLM__CALL_TRANSCRIPTS_ALL"."DATE_CREATED" AS "DATE_CREATED",
      "LLM__CALL_TRANSCRIPTS_ALL"."LANGUAGE" AS "LANGUAGE",
      "LLM__CALL_TRANSCRIPTS_ALL"."COUNTRY" AS "COUNTRY",
      "LLM__CALL_TRANSCRIPTS_ALL"."PRODUCT" AS "PRODUCT",
      "LLM__CALL_TRANSCRIPTS_ALL"."CATEGORY" AS "CATEGORY",
      "LLM__CALL_TRANSCRIPTS_ALL"."DAMAGE_TYPE" AS "DAMAGE_TYPE",
      "LLM__CALL_TRANSCRIPTS_ALL"."TRANSCRIPT_SENTIMENT" AS "TRANSCRIPT_SENTIMENT",
      "TRANSCRIPT_CUSTOMER".value:"answer"::STRING AS "CUSTOMER_NAME"
    FROM
       "DATABASE_NAME"."SCHEMA_NAME"."LLM__CALL_TRANSCRIPTS_ALL" "LLM__CALL_TRANSCRIPTS_ALL",
      lateral flatten(input => "TRANSCRIPT_CUSTOMER", OUTER => TRUE) "TRANSCRIPT_CUSTOMER"
  );

### Conclusion
In this walkthrough, you have seen how Coalesce automatically generates code to seamlessly build data pipelines on Snowflake. Using the Coalesce, users can defer the need of writing all of the code found here, and instead, focus on the business impact of the data products they are building. 